# Exploratory & A/B Testing 

## 1. Subject Line A/B Performance

* Q1: Do variant A vs B subject lines produce statistically significant differences in open rate?

* Q2:  Is the effect of subject variant on open rate consistent across segments and channels, or explained by confounders

## 2. Timing & Frequency Effects

* Q3: Does send time (hour/day) affect open/CTR?

* Q4: How does customer engagement fatigue—measured by cumulative opens and inactivity periods—influence unsubscribe rates and purchase likelihood?

## 3. Segment-Level Insights

* Q5: Which customer segments (RFM quintiles, new vs. repeat) respond best to promotions vs. newsletters?

* Q6: What is the interplay between purchase frequency and email engagement?

* Q7: For engaged customers (opened/clicked), what is the uplift in next-30-day order rate and average order value vs. non-openers?

In [18]:
import pandas as pd
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
email_campaign_df = pd.read_csv('../data/email_campaigns.csv')
for col in ['campaign_id', 'channel', 'segment', 'customer_id', 'recipient_name', 'send_timestamp', 'subject_variant', 'subject_line', 'device']:
    email_campaign_df[col] = email_campaign_df[col].astype('string')

email_engagement_df = pd.read_csv('../data/email_engagement.csv')
for col in ['campaign_id', 'customer_id', 'send_timestamp', 'subject_variant']:
    email_engagement_df[col] = email_engagement_df[col].astype('string')




# Convert timestamps
email_campaign_df['send_timestamp'] = pd.to_datetime(email_campaign_df['send_timestamp'], errors='coerce')
email_engagement_df['send_timestamp'] = pd.to_datetime(email_engagement_df['send_timestamp'], errors='coerce')

In [20]:
# 10% sample of 10 million rows 
email_campaign_df = email_campaign_df.sample(frac=0.10, random_state=42)
filtered_keys = email_campaign_df[['campaign_id', 'customer_id', 'send_timestamp']]

email_engagement_df = email_engagement_df.merge(
    filtered_keys,
    on=['campaign_id', 'customer_id', 'send_timestamp'],
    how='inner'
)


In [21]:
email_engagement_df.describe()

,send_timestamp,opened,clicked,unsubscribed,purchase,revenue
count,1568632,1.568632e+06,1.568632e+06,1.568632e+06,1.568632e+06,1.568632e+06
mean,2025-01-06 06:16:27.767429888,1.100730e-01,4.000301e-03,3.602502e-03,4.666486e-04,1.622352e-02
min,2024-07-01 10:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2024-10-08 10:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2024-12-24 10:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2025-04-16 10:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2025-06-30 19:00:00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.352831e+02
std,NaN,3.129808e-01,6.312132e-02,5.991266e-02,2.159702e-02,1.137975e+00


## 1. Subject Line A/B Performance

### Q1: Do variant A vs B subject lines produce statistically significant differences in open rate?

#### Statistical test (e.g., two-proportion z-test) for significance.

In [22]:
from statsmodels.stats.proportion import proportions_ztest


# Aggregate open counts by variant 

agg= (email_engagement_df
      .groupby('subject_variant')['opened']
      .agg(['sum','count'])
      )


## Variant A vs Variant B

open = agg.loc[['A','B'],'sum'] 

totals = agg.loc[['A','B'],'count']


# two-propertion z test
stats, pval = proportions_ztest(open,totals)

print(f"Z-Stat: {stats}, p-value: {pval}")



Z-Stat: -13.157596463000761, p-value: 1.539042593237991e-39


#### Effect size calculation (difference in open rate) for praticial impact

In [23]:
from statsmodels.stats.proportion import confint_proportions_2indep

#open rates 
rate_A = open.iloc[0]/totals.iloc[0]
rate_B = open.iloc[1]/totals.iloc[1]

effect_size= rate_A-rate_B


## confidence interval for difference 
# method = wald or score can be used 

lower,upper = confint_proportions_2indep(open.iloc[0],totals.iloc[0],open.iloc[1],totals.iloc[1],method='score')

print(f"Effect size (A-B): {effect_size:5f}")

print(f"95% CI: ({lower:.5f}, {upper:.5f})")



Effect size (A-B): -0.008399
95% CI: (-0.00965, -0.00715)


### *Summary for Subject Line A/B Performance*

The statistical analysis shows a highly significant difference in open rates between subject line variants A and B.

- Two-proportion z-test result: Z = -32.86, p-value ≈ 0, indicating the difference is not due to chance.
- Effect size (A - B): -0.00937, meaning variant A's open rate is approximately 0.9% lower than variant B's.
-  95% confidence interval for the difference: (-0.00993, -0.00881), confirming precision and that the difference is consistently below zero.

Conclusion: Subject line variant has a statistically significant and measurable impact on open rate. The difference is precise and robust given the large dataset.

#### Q2: Is the effect of subject variant on open rate consistent across segments and channels, or explained by confounders

In [24]:
# Merge email_campaign_df with email_engagement_df to add channel and segment info to engagement data
# Drop duplicates to keep unique campaign_id with channel and segment
campaign_agg = email_campaign_df[['campaign_id', 'channel', 'segment']].drop_duplicates(subset=['campaign_id']).copy()

# Merge aggregated campaign info with engagement data
merged_df = email_engagement_df.merge(campaign_agg, on='campaign_id', how='left')



In [15]:
merged_df.columns

Index(['campaign_id', 'customer_id', 'send_timestamp', 'subject_variant',
       'opened', 'clicked', 'unsubscribed', 'purchase', 'revenue', 'channel',
       'segment'],
      dtype='object')

 #### logistic regression to model open probability

In [25]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

## prepare merged df with nesessary columns 
## ensuring categorical variables are treated as such 

merged_df['subject_variant']= merged_df['subject_variant'].astype('category')
merged_df['channel']= merged_df['channel'].astype('category')
merged_df['segment']= merged_df['segment'].astype('category')


# Logistic regression formula with interactions
formula = 'opened ~ subject_variant + channel + segment + subject_variant:channel + subject_variant:segment'


# fit model using statsmodel 
model = smf.logit(formula=formula, data=merged_df).fit()

print(model.summary())


Optimization terminated successfully.
         Current function value: 0.353403
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 opened   No. Observations:               987776
Model:                          Logit   Df Residuals:                   987756
Method:                           MLE   Df Model:                           19
Date:                Fri, 08 Aug 2025   Pseudo R-squ.:                0.001026
Time:                        11:53:44   Log-Likelihood:            -3.4908e+05
converged:                       True   LL-Null:                   -3.4944e+05
Covariance Type:            nonrobust   LLR p-value:                9.891e-140
                                                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                           

#### *Model Significance Order and Conclusion*

Order of predictors by statistical significance (lowest to highest p-value):

1. channel[T.promo] (p ≈ 0.000, z=5.933) - strongest positive effect  
2. channel[T.loyalty] (p ≈ 0.000, z=4.759)  
3. channel[T.transactional] (p=0.004, z=2.897)  
4. subject_variant[T.B] (p=0.008, z=2.656)  
5. channel[T.survey] (p=0.055, z=1.921) - borderline  
6. channel[T.newsletter] (p=0.265, z=1.119) - not significant  
7. segment[...] terms – all p > 0.1, not significant  
8. interaction terms – all p > 0.1, not significant  

**Conclusion:**  

The promotional, loyalty, and transactional channels significantly increase the odds of email opens, with promo having the largest effect.

Subject variant B also significantly increases open likelihood but with a smaller effect than these top channels.

All segment and interaction effects are statistically insignificant, while the newsletter and survey channels show weak or borderline impact on open rates.


## 2. Timing & Frequency Effects

### Q3: Does send time (hour/day) affect open/CTR?

In [26]:
grouped = (merged_df
           .groupby([merged_df['send_timestamp'].dt.dayofweek.rename('day_sent'),
                     merged_df['send_timestamp'].dt.hour.rename('hour_sent')])
           .agg(opened_sum=('opened', 'sum'),
                clicked_sum=('clicked', 'sum'),
                email_sent=('campaign_id', 'count'))
          ).reset_index()

grouped['open_rate'] = grouped['opened_sum'] / grouped['email_sent']
grouped['ctr'] = grouped['clicked_sum'] / grouped['email_sent']



#### Logistic regression to test impact of send day and hour on email open probability

In [27]:
merged_df['day_sent']= merged_df['send_timestamp'].dt.dayofweek
merged_df['hour_sent']= merged_df['send_timestamp'].dt.hour


logit_df = merged_df[['opened', 'day_sent', 'hour_sent']].copy()

In [28]:
import numpy as np
import statsmodels.formula.api as smf

# Random 1% sample (≈ 100k rows) to fit model
sample_df = merged_df.sample(frac=0.01, random_state=42)

# Convert to categorical
sample_df['day_sent'] = sample_df['day_sent'].astype('category')
sample_df['hour_sent'] = sample_df['hour_sent'].astype('category')

# Fit logistic regression
formula = 'opened ~ day_sent + hour_sent'
model = smf.logit(formula=formula, data=sample_df)
result = model.fit()

print(result.summary())



Optimization terminated successfully.
         Current function value: 0.339970
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 opened   No. Observations:                15686
Model:                          Logit   Df Residuals:                    15668
Method:                           MLE   Df Model:                           17
Date:                Fri, 08 Aug 2025   Pseudo R-squ.:                0.002885
Time:                        11:56:08   Log-Likelihood:                -5332.8
converged:                       True   LL-Null:                       -5348.2
Covariance Type:            nonrobust   LLR p-value:                   0.02079
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.9455      0.199     -9.778      0.000      -2.335      -1.555
day_sent[T.1] 

#### Logistic regression to test impact of send day and hour on email CTR probability

In [29]:

import statsmodels.api as sm

grouped['failures'] = grouped['email_sent'] - grouped['clicked_sum']

grouped['day_sent'] = grouped['day_sent'].astype('category')
grouped['hour_sent'] = grouped['hour_sent'].astype('category')

formula = 'clicked_sum + failures ~ day_sent + hour_sent'

model = smf.glm(formula=formula, data=grouped, family=sm.families.Binomial())
result = model.fit()

print(result.summary())


                      Generalized Linear Model Regression Results                      
Dep. Variable:     ['clicked_sum', 'failures']   No. Observations:                   44
Model:                                     GLM   Df Residuals:                       26
Model Family:                         Binomial   Df Model:                           17
Link Function:                           Logit   Scale:                          1.0000
Method:                                   IRLS   Log-Likelihood:                -162.36
Date:                         Fri, 08 Aug 2025   Deviance:                       35.672
Time:                                 11:57:08   Pearson chi2:                     35.6
No. Iterations:                              9   Pseudo R-squ. (CS):             0.9974
Covariance Type:                     nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------

#### *Summary for open and CTR probability*

Conclusion: The model's low pseudo R-squared (0.0025) and coefficients show weak predictive power and inconsistent day/hour effects, suggesting the data is noisy, synthetic, or poorly recorded. The lack of strong positive coefficients means no clear best send time emerges. The dataset or feature engineering likely needs improvement before actionable insights. The data shows Sunday at 00:00 being the most optimal day to send the emails which makes no sense in the real world.

In [37]:
merged_df.columns

Index(['campaign_id', 'customer_id', 'send_timestamp', 'subject_variant',
       'opened', 'clicked', 'unsubscribed', 'purchase', 'revenue', 'channel',
       'segment', 'day_sent', 'hour_sent'],
      dtype='object')

## Q4: How does customer engagement fatigue-measured by cumulative opens and inactivity periods-influence unsubscribe rates and purchase likelihood?

In [30]:
df = merged_df.copy()

In [31]:

df['customer_id'] = df['customer_id'].astype('category')

df['purchase'] = df['purchase'].astype('int8')
df['send_timestamp'] = pd.to_datetime(df['send_timestamp'])

## sort the values by customer_id and send timestamp
df.sort_values(['customer_id', 'send_timestamp'], inplace=True)

## identify first row per customer 
df['is_first']= df.groupby('customer_id',observed=True).cumcount()==0

## has any prior purchase? 
df['prior_purchase']= ( 
    df.groupby('customer_id',observed=True)['purchase']
    .transform(lambda x: x.shift().cummax().fillna(0)))

## assing segments 
df['segment']='inactive'
df.loc[df['is_first'],'segment']= 'new'

# if customer is not new and has prior purchase then they are a repat  
df.loc[(~df['is_first']) & (df['prior_purchase']>0),'segment']='repeat'

In [32]:
df['days_since_last_purchase'] = df.groupby('customer_id',observed=True)['send_timestamp'].diff().dt.days.fillna(0)

df['days_since_last_purchase_cum'] = df.groupby('customer_id',observed=True)['days_since_last_purchase'].cumsum()


In [33]:
import statsmodels.formula.api as smf

# Create per-customer cumulative opens
df['cumulative_opens'] = (
    df.groupby('customer_id', observed=True)['opened']
      .cumsum()
)

# Scale continuous predictors to avoid overflow
df['cumulative_opens_scaled'] = (
    (df['cumulative_opens'] - df['cumulative_opens'].mean()) /
    df['cumulative_opens'].std()
)
df['days_since_last_purchase_cum_scaled'] = (
    (df['days_since_last_purchase_cum'] - df['days_since_last_purchase_cum'].mean()) /
    df['days_since_last_purchase_cum'].std()
)

# Stratified sample for unsubscribe model
sample_df_unsub = (
    df.groupby('unsubscribed', group_keys=False, observed=True)
      .sample(n=min(50000, df['unsubscribed'].value_counts().min()),
              random_state=42)
)

# Stratified sample for purchase model
sample_df_purchase = (
    df.groupby('purchase', group_keys=False, observed=True)
      .sample(n=min(50000, df['purchase'].value_counts().min()),
              random_state=42)
)

# Ensure binary outcomes are int
sample_df_unsub['unsubscribed'] = sample_df_unsub['unsubscribed'].astype(int)
sample_df_purchase['purchase'] = sample_df_purchase['purchase'].astype(int)

# Logistic regression: unsubscribe ~ fatigue/inactivity
model_unsub = smf.logit(
    formula='unsubscribed ~ days_since_last_purchase_cum_scaled + cumulative_opens_scaled + segment',
    data=sample_df_unsub
).fit()
print(model_unsub.summary())




Optimization terminated successfully.
         Current function value: 0.681011
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:           unsubscribed   No. Observations:                11302
Model:                          Logit   Df Residuals:                    11297
Method:                           MLE   Df Model:                            4
Date:                Fri, 08 Aug 2025   Pseudo R-squ.:                 0.01751
Time:                        11:58:35   Log-Likelihood:                -7696.8
converged:                       True   LL-Null:                       -7833.9
Covariance Type:            nonrobust   LLR p-value:                 3.723e-58
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              -0.0469      0.020 

#### *Logistic regression: unsubscribe summary*

* days_since_last_purchase_cum_scaled has a strong negative coefficient (p<0.001).  
* Longer inactivity periods correlate with a significantly lower likelihood of unsubscribing.  
* cumulative_opens_scaled has a strong positive coefficient (p<0.001). Customers with more cumulative opens are significantly more likely to unsubscribe.  
* Segment type (new vs repeat) is not significant.  

Conclusion: Engagement fatigue is evident. Frequent openers face higher unsubscribe risk, while longer inactivity correlates with reduced risk. Purchase recency appears less influential than cumulative engagement.  




In [34]:
# Logistic regression: purchase ~ fatigue/inactivity
import statsmodels.formula.api as smf

# Collapse 'new' into 'inactive'
sample_df_purchase['segment'] = sample_df_purchase['segment'].replace({'new': 'inactive'})

# Scale predictors
for col in ['days_since_last_purchase_cum', 'cumulative_opens']:
    sample_df_purchase[col + '_scaled'] = (
        sample_df_purchase[col] - sample_df_purchase[col].mean()
    ) / sample_df_purchase[col].std()

# Fit logistic regression
model_purchase = smf.logit(
    formula='purchase ~ days_since_last_purchase_cum_scaled + cumulative_opens_scaled + C(segment)',
    data=sample_df_purchase
).fit()
print(model_purchase.summary())




Optimization terminated successfully.
         Current function value: 0.589106
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               purchase   No. Observations:                 1464
Model:                          Logit   Df Residuals:                     1460
Method:                           MLE   Df Model:                            3
Date:                Fri, 08 Aug 2025   Pseudo R-squ.:                  0.1501
Time:                        12:03:18   Log-Likelihood:                -862.45
converged:                       True   LL-Null:                       -1014.8
Covariance Type:            nonrobust   LLR p-value:                 9.885e-66
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              -0.0011      0.061 

#### *Logistic regression: purchase summary*

* cumulative_opens_scaled has a strong positive effect (coef=0.9183, p<0.001).  
* Customers with more cumulative opens are significantly more likely to make a purchase.  
* Segment repeat has a positive and significant effect (coef=0.8306, p=0.004). Repeat customers are more likely to purchase than new customers.  
* days_since_last_purchase_cum_scaled is positive but not statistically significant (p=0.159).  

Conclusion: Cumulative engagement strongly drives purchase likelihood. Repeat customers have a higher purchase probability. Purchase recency shows no clear effect here.


## 3. Segment-Level Insights

#### Q5: Which customer segments (RFM quintiles, new vs. repeat) respond best to promotions vs. newsletters?

In [35]:
# Create or confirm RFM columns
df['recency'] = df['days_since_last_purchase']  # lower better
df['frequency'] = df['prior_purchase']  # higher better
df['monetary'] = df['revenue']  # higher better

# For recency, invert so higher is better for qcut
df['recency_inv'] = -df['recency']  # invert to sort recent high

# Define quantile labels
quantile_labels = [1, 2, 3, 4, 5]

df['R_rank'] = df['recency_inv'].rank(method='first')
df['R_quintile'] = pd.qcut(df['R_rank'], q=5, labels=quantile_labels, duplicates='drop')

df['F_rank'] = df['frequency'].rank(method='first')
df['F_quintile'] = pd.qcut(df['F_rank'], q=5, labels=quantile_labels, duplicates='drop')

df['M_rank'] = df['monetary'].rank(method='first')
df['M_quintile'] = pd.qcut(df['M_rank'], q=5, labels=quantile_labels, duplicates='drop')

df['RFM_score'] = df['R_quintile'].astype(str) + df['F_quintile'].astype(str) + df['M_quintile'].astype(str)



In [38]:
# Filter for promo and newsletter channels only

df_filtered = df[df['channel'].isin(['promo', 'newsletter'])].copy()


In [40]:
import pandas as pd


# Remove rows where segment == 'new' and clicked == 0
df_filtered_nonzero = df_filtered[~((df_filtered['segment'] == 'new') & (df_filtered['clicked'] == 0))].copy()

# Ensure categorical types with limited categories
df_filtered_nonzero['channel'] = pd.Categorical(df_filtered_nonzero['channel'], categories=['promo', 'newsletter'])
df_filtered_nonzero['segment'] = df_filtered_nonzero['segment'].astype('category')
df_filtered_nonzero['rfm_quintile'] = df_filtered_nonzero['RFM_score'].astype('category')

# Group by segment, channel, and rfm_quintile aggregating clicks and counts
group_cols = ['segment', 'channel', 'rfm_quintile']
grouped = (
    df_filtered_nonzero
    .groupby(group_cols, observed=True)['clicked']
    .agg(['sum', 'count'])
    .reset_index()
)

# Calculate click rate per group
grouped['click_rate'] = grouped['sum'] / grouped['count']




In [41]:

import statsmodels.api as sm
# Filter out perfect 0 or 1 click rates to avoid separation in logistic regression
grouped = grouped[(grouped['click_rate'] > 0) & (grouped['click_rate'] < 1)].copy()

# One-hot encode categorical predictors, drop first to avoid multicollinearity
X = pd.get_dummies(grouped[['segment', 'channel', 'rfm_quintile']], drop_first=True)

# Add intercept constant
X = sm.add_constant(X).astype(float)

# Response: number of clicks
y = grouped['sum'] / grouped['count']


# Frequency weights: number of emails sent per group
weights = grouped['count'].astype(float)

# Fit binomial GLM with frequency weights
model = sm.GLM(y, X, family=sm.families.Binomial(), freq_weights=weights)
result = model.fit()

print(result.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  106
Model:                            GLM   Df Residuals:                  1314151
Model Family:                Binomial   Df Model:                           52
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -27473.
Date:                Fri, 08 Aug 2025   Deviance:                       32.433
Time:                        12:06:24   Pearson chi2:                     32.5
No. Iterations:                   100   Pseudo R-squ. (CS):             0.9901
Covariance Type:            nonrobust                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -5.7904      0

#### *Q5: Customer Segment Response by Channel Type — Promotions vs. Newsletters*

Analysis confirms that customer segment (new vs. repeat) significantly affects click rates, with repeat customers showing higher engagement. Channel type also has a significant impact: newsletters perform worse than promotions, as indicated by the negative coefficient for the newsletter channel.

Therefore, while segment drives response differences, the data supports prioritizing promotional emails over newsletters to maximize click rates within these customer segments.


### Q6: What is the interplay between purchase frequency and email engagement?

In [42]:



df_filtered.columns

## group by purchase frequency 

Index(['campaign_id', 'customer_id', 'send_timestamp', 'subject_variant',
       'opened', 'clicked', 'unsubscribed', 'purchase', 'revenue', 'channel',
       'segment', 'day_sent', 'hour_sent', 'is_first', 'prior_purchase',
       'days_since_last_purchase', 'days_since_last_purchase_cum',
       'cumulative_opens', 'cumulative_opens_scaled',
       'days_since_last_purchase_cum_scaled', 'recency', 'frequency',
       'monetary', 'recency_inv', 'R_rank', 'R_quintile', 'F_rank',
       'F_quintile', 'M_rank', 'M_quintile', 'RFM_score'],
      dtype='object')

In [43]:
grouped_pf_engagement = (
    df_filtered
    .groupby(['F_quintile', 'clicked'],observed=True)
    .agg(
        count_emails=('clicked', 'count'),
        clicks=('clicked', 'sum'),
        avg_click_rate=('clicked', 'mean'),
        avg_revenue=('revenue', 'mean')
    )
    .reset_index()
)





In [44]:
import scipy.stats as stats
import pandas as pd

# Contingency table of frequency quintile vs clicked
contingency = pd.crosstab(df_filtered['F_quintile'], df_filtered['clicked'])

# Chi-square test of independence
chi2, p_value, dof, expected = stats.chi2_contingency(contingency)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p_value}")


Chi-square statistic: 35.62945887912943
P-value: 3.448730717220716e-07




The p-value (≈ 3.45e-7) is effectively zero, indicating a statistically significant association between purchase frequency and email engagement.

In [45]:
# ANOVA test on revenue by F_quintile groups
groups = [group['revenue'].dropna() for name, group in df_filtered.groupby('F_quintile',observed=True)]
anova_stat, anova_p = stats.f_oneway(*groups)

print(f"ANOVA F-statistic: {anova_stat}")
print(f"P-value: {anova_p}")


ANOVA F-statistic: 4.2482048127942456
P-value: 0.0019392522774433275


The ANOVA p-value is 0.0019, below 0.05, indicating a statistically significant difference in average revenue across purchase frequency groups


#### *Q6: Relationship Between Purchase Frequency and Email Engagement*

Statistical analysis reveals a significant difference in average revenue across purchase frequency groups, indicating that customers with varying purchase frequencies generate different revenue levels. Email engagement (click behavior) also varies significantly by purchase frequency, showing differentiated response patterns.

*Conclusion:*  
* Purchase frequency is significantly associated with both email engagement and average revenue.  
* Higher purchase frequency customers tend to generate more revenue and engage more with emails.  
* This analysis does not establish causation between email engagement and purchases.

*Further analysis with causal or temporal models is necessary to determine if email engagement influences purchase behavior.


### Q7: For engaged customers (opened/clicked), what is the uplift in next-30-day order rate and average order value vs. non-openers?

In [61]:
import pandas as pd
from datetime import timedelta

df_filtered['engaged'] = (df_filtered['opened'] | df_filtered['clicked']).astype(int)
df_filtered['window_end'] = df_filtered['send_timestamp'] + timedelta(days=30)
df_filtered['customer_id'] = df_filtered['customer_id'].astype(str)

purchases = df_filtered[df_filtered['purchase'] == 1][['customer_id', 'send_timestamp', 'revenue']]
purchases = purchases.rename(columns={'send_timestamp': 'purchase_timestamp', 'revenue': 'purchase_revenue'})
purchases['customer_id'] = purchases['customer_id'].astype(str)

# Cross join filter: join on customer_id where purchase_timestamp between send_timestamp and window_end
merged = pd.merge(df_filtered, purchases, on='customer_id', how='left')

mask = (merged['purchase_timestamp'] >= merged['send_timestamp']) & (merged['purchase_timestamp'] <= merged['window_end'])
merged = merged[mask]

# Aggregate per email row
agg = merged.groupby(['customer_id', 'send_timestamp', 'engaged']).agg(
    total_orders=('purchase_timestamp', 'count'),
    total_revenue=('purchase_revenue', 'sum')
).reset_index()

# Join agg back to emails without purchases, fill 0
emails = df_filtered[['customer_id', 'send_timestamp', 'engaged']].drop_duplicates()
result = emails.merge(agg, on=['customer_id', 'send_timestamp', 'engaged'], how='left').fillna({'total_orders':0, 'total_revenue':0})

summary = result.groupby('engaged').agg(
    total_emails=('send_timestamp', 'count'),
    total_orders=('total_orders', 'sum'),
    total_revenue=('total_revenue', 'sum')
).reset_index()

summary['order_rate'] = summary['total_orders'] / summary['total_emails']
summary['avg_order_value'] = summary.apply(
    lambda row: row['total_revenue'] / row['total_orders'] if row['total_orders'] > 0 else 0, axis=1
)

order_rate_uplift = summary.loc[summary['engaged'] == 1, 'order_rate'].values[0] - summary.loc[summary['engaged'] == 0, 'order_rate'].values[0]
avg_order_value_uplift = summary.loc[summary['engaged'] == 1, 'avg_order_value'].values[0] - summary.loc[summary['engaged'] == 0, 'avg_order_value'].values[0]

print(f"Next-30-day order rate (non-engaged): {summary.loc[summary['engaged'] == 0, 'order_rate'].values[0]:.4f}")
print(f"Next-30-day order rate (engaged): {summary.loc[summary['engaged'] == 1, 'order_rate'].values[0]:.4f}")
print(f"Order rate uplift: {order_rate_uplift:.4f}")

print(f"Avg order value (non-engaged): £{summary.loc[summary['engaged'] == 0, 'avg_order_value'].values[0]:.2f}")
print(f"Avg order value (engaged): £{summary.loc[summary['engaged'] == 1, 'avg_order_value'].values[0]:.2f}")
print(f"Avg order value uplift: £{avg_order_value_uplift:.2f}")


Next-30-day order rate (non-engaged): 0.0079
Next-30-day order rate (engaged): 0.0144
Order rate uplift: 0.0065
Avg order value (non-engaged): £36.03
Avg order value (engaged): £35.60
Avg order value uplift: £-0.43


#### Q7: Impact of Email Engagement on Next-30-Day Order Rate and Average Order Value

Engaged customers (opened or clicked emails) show a higher next-30-day order rate (1.44%) compared to non-engaged customers (0.79%), an uplift of 0.65%. However, the average order value for engaged customers (£35.60) is slightly lower than for non-engaged customers (£36.03).

This suggests engaged customers purchase more frequently but at marginally lower values per order, likely influenced by promotional offers. Non-engaged customers buy less often but spend more per transaction, potentially at regular prices.

Further analysis of pricing and discount application is needed to validate this interpretation.
